# Lightweight Fine-Tuning Project

TODO: In this cell, describe your choices for each of the following

* PEFT technique: 
* Model: 
* Evaluation approach: 
* Fine-tuning dataset: 

## Loading and Evaluating a Foundation Model

TODO: In the cells below, load your chosen pre-trained Hugging Face model and evaluate its performance prior to fine-tuning. This step includes loading an appropriate tokenizer and dataset.

In [9]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from transformers import Trainer, TrainingArguments, DataCollatorWithPadding
import numpy as np
from datasets import load_dataset

model = AutoModelForSequenceClassification.from_pretrained("distilbert/distilbert-base-uncased")
tokenizer = AutoTokenizer.from_pretrained("distilbert/distilbert-base-uncased")

dataset = load_dataset("agentlans/grammar-classification")
dataset["train"][0]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert/distilbert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


{'text': 'We have carried out a transaction cost analysis to help us understand the quality of execution on the execution venues where we execute/place orders.',
 'grammar': 1}

In [10]:
def tokenizing_map(dataset_entry):
    return tokenizer(dataset_entry["text"], padding="max_length", truncation=True)

# temp small datasets to quickly iterate on config
small_validation_dataset = dataset["validation"].select(range(1000)).rename_column("grammar","labels")
small_train_dataset = dataset["train"].select(range(1000)).rename_column("grammar","labels")

small_validation_dataset=small_validation_dataset.map(tokenizing_map, batched=True)
small_train_dataset=small_train_dataset.map(tokenizing_map, batched=True)


for param in model.base_model.parameters():
    param.requires_grad = False
    
small_train_dataset

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Dataset({
    features: ['text', 'labels', 'input_ids', 'attention_mask'],
    num_rows: 1000
})

In [11]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return { "accuracy": (predictions == labels).mean() }

trainer = Trainer(
    model=model,
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
    args=TrainingArguments(
        output_dir="./data/grammar",
        evaluation_strategy="epoch",
        num_train_epochs=3
    ),
    train_dataset=small_train_dataset,
    eval_dataset=small_validation_dataset,
    compute_metrics=compute_metrics
)
trainer.train()
trainer.evaluate()

You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.688315,0.514000
2,No log,0.674166,0.610000
3,No log,0.674905,0.584000


{'eval_loss': 0.6749048829078674,
 'eval_accuracy': 0.584,
 'eval_runtime': 15.5316,
 'eval_samples_per_second': 64.385,
 'eval_steps_per_second': 8.048,
 'epoch': 3.0}

## Performing Parameter-Efficient Fine-Tuning

TODO: In the cells below, create a PEFT model from your loaded model, run a training loop, and save the PEFT model weights.

In [12]:
from peft import LoraConfig, get_peft_model
config = LoraConfig(
    task_type="CAUSAL_LM",
    target_modules=["q_lin","k_lin","v_lin"]  ,
)
print(model)
lora_model = get_peft_model(model, config)


DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
 

In [13]:
from transformers import default_data_collator
lora_trainer = Trainer(
    model=lora_model,
    data_collator=default_data_collator,
    args=TrainingArguments(
        output_dir="./data/grammar",
        evaluation_strategy="epoch",
        num_train_epochs=3
    ),
    train_dataset=small_train_dataset,
    eval_dataset=small_validation_dataset,
    compute_metrics=compute_metrics
)
lora_model.print_trainable_parameters()
# print ( lora_trainer.train_dataset[0])
# print((small_train_dataset[0]))
# print((small_validation_dataset[0]))
lora_trainer.train()
lora_trainer.evaluate()

trainable params: 221,184 || all params: 67,176,194 || trainable%: 0.3292594992803552
{'text': 'We have carried out a transaction cost analysis to help us understand the quality of execution on the execution venues where we execute/place orders.', 'labels': 1, 'input_ids': [101, 2057, 2031, 3344, 2041, 1037, 12598, 3465, 4106, 2000, 2393, 2149, 3305, 1996, 3737, 1997, 7781, 2006, 1996, 7781, 9356, 2073, 2057, 15389, 1013, 2173, 4449, 1012, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.672922,0.595000
2,No log,0.669950,0.596000
3,No log,0.669026,0.598000


{'eval_loss': 0.6690261363983154,
 'eval_accuracy': 0.598,
 'eval_runtime': 16.4616,
 'eval_samples_per_second': 60.747,
 'eval_steps_per_second': 7.593,
 'epoch': 3.0}

###  ⚠️ IMPORTANT ⚠️

Due to workspace storage constraints, you should not store the model weights in the same directory but rather use `/tmp` to avoid workspace crashes which are irrecoverable.
Ensure you save it in /tmp always.

In [ ]:
# Saving the model
44model.save("/tmp/your_model_name")

## Performing Inference with a PEFT Model

TODO: In the cells below, load the saved PEFT model weights and evaluate the performance of the trained PEFT model. Be sure to compare the results to the results from prior to fine-tuning.